In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.baseline_rank.baseline_rank import BaselineRank
from scripts.models.ltr.train import LTRTrain
from scripts.extractive_summary.ltr.ltr_features_targets import LTRFeaturesTargets
from scripts.models.ltr_gbt.train import LTRGBTTrain
from scripts.models.ltr_svm_tf.train import LTRSVMTFTrain
from scripts import conf
%reload_ext autoreload

In [4]:
import pandas as pd

# Evaluación

Se usará el mismo desarrollo que se utilizó cuando se calcularon las métricas de los resúmenes generados por eventos clave y por grafos
conceptuales

# Baseline

## Carga de datos

In [17]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'rouge'
drop_teams = True
lemma = True
metric_params = {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [18]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to rouge


In [19]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [20]:
rank.run()

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/baseline_rank/60099ac54d/summaries.csv


In [29]:
pd_summaries = rank.read_summaries()

Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/60099ac54d/summaries.csv


In [23]:
articles_path = '{}articles_events.csv'.format(CSV_DATA_PATH)

In [26]:
pd_matches = pd.read_csv(articles_path)

In [34]:
join_cols = ['url']
match_cols = join_cols + ['json_file', 'article', 'events']
sum_cols = join_cols + ['summary_events']

In [35]:
pd_matches_sum = pd_matches[match_cols].merge(pd_summaries[sum_cols], on=join_cols, how='inner')

In [36]:
pd_matches_sum

,url,json_file,article,events,summary_events
0,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,A goal and an assist from Riyad Mahrez sealed ...,Attempt blocked. Ilkay Gündogan (Manchester Ci...,Attempt saved. Riyad Mahrez (Manchester City) ...
1,http://www.premierleague.com/match/38679,premier_league_2018_2019.json,A second-half double by Pierre-Emerick Aubamey...,Hand ball by Tom Heaton (Burnley). Yellow Card...,Pierre-Emerick Aubameyang (Arsenal) hits the r...
2,http://www.premierleague.com/match/38680,premier_league_2018_2019.json,Michy Batshuayi's brace helped Crystal Palace ...,Andros Townsend (Crystal Palace) wins a free k...,Andros Townsend (Crystal Palace) wins a free k...
3,http://www.premierleague.com/match/38681,premier_league_2018_2019.json,Fulham were overpowered by an impressive Newca...,Attempt missed. Jonjo Shelvey (Newcastle Unite...,Attempt missed. Jonjo Shelvey (Newcastle Unite...
4,http://www.premierleague.com/match/38682,premier_league_2018_2019.json,Chelsea confirmed a third-place finish after a...,Foul by Jorginho (Chelsea). Jamie Vardy (Leice...,Jamie Vardy (Leicester City) wins a free kick ...
...,...,...,...,...,...
4518,https://www.espn.com/soccer/report?gameId=522712,italian_serie_a_2018_2019.json,Inter Milan notched their seventh successive S...,"Corner, Inter Milan. Conceded by Cristian Rom...",Attempt missed. Ivan Perisic (Inter Milan) rig...
4519,https://www.espn.com/soccer/report?gameId=522672,italian_serie_a_2018_2019.json,Two goals from Arkadiusz Milik helped Napoli m...,"Offside, Napoli. Elseid Hysaj tries a through ...","Goal! Napoli 1, Frosinone 0. Piotr Zielinski ..."
4520,https://www.espn.com/soccer/report?gameId=522701,italian_serie_a_2018_2019.json,Inter Milan's run of seven successive league w...,Attempt blocked. Hans Hateboer (Atalanta) head...,"Corner, Atalanta. Conceded by Samir Handanovi..."
4521,https://www.espn.com/soccer/report?gameId=522667,italian_serie_a_2018_2019.json,Inter Milan rebounded from the disappointment ...,"Corner, Inter Milan. Conceded by Bram Nuytinc...",Marco D'Alessandro (Udinese) wins a free kick ...


## Evaluación

In [37]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [38]:
file_name = 'baseline_rank'

In [44]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [47]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=False)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_avg.pickle


In [48]:
avg_scores_dict

{'rouge-1': {'f': 0.2174203202733838,
  'p': 0.2446300614718203,
  'r': 0.2258641620482864},
 'rouge-2': {'f': 0.046197910241048024,
  'p': 0.05345303695536832,
  'r': 0.047444265068684204},
 'rouge-l': {'f': 0.2009559760580376,
  'p': 0.3273697532121361,
  'r': 0.1551321076144749}}

### Con procesado

In [49]:
file_name = 'baseline_rank_processed'

In [50]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [53]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=True)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_processed_avg.pickle


In [54]:
avg_scores_dict

{'rouge-1': {'f': 0.21470774269529114,
  'p': 0.22181043816460994,
  'r': 0.24382593800687752},
 'rouge-2': {'f': 0.06066140105765257,
  'p': 0.06151992918019337,
  'r': 0.07034246895093633},
 'rouge-l': {'f': 0.260602313959701,
  'p': 0.3969169606060778,
  'r': 0.20847736563413624}}

# Random forest

## Carga de datos

In [55]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1]}
num_features = ['tfidf_sum']
model_params = {
    'n_estimators': 100,
    'max_depth': 2,
    'bootstrap': True,  
}

In [56]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to rouge


In [57]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [60]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/9ef9119130'

In [58]:
rank_model.run()

Categorizing n_players...
Computing new length...
Dropping {'n_players_cat', 'score', 'sentence_ix', 'event_ix', 'sim_previous_3', 'sim_previous_1', 'advantage', 'url', 'length', 'equalize', 'players_importance', 'total_length', 'sim_previous_5', 'json_file', 'n_stop', 'n_players', 'position'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/9ef9119130/summaries.csv


In [61]:
pd_summaries = rank_model.read_summaries()

Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/9ef9119130/summaries.csv


In [62]:
articles_path = '{}articles_events.csv'.format(CSV_DATA_PATH)

In [63]:
pd_matches = pd.read_csv(articles_path)

In [64]:
join_cols = ['url']
match_cols = join_cols + ['json_file', 'article', 'events']
sum_cols = join_cols + ['summary_events']

In [65]:
pd_matches_sum = pd_matches[match_cols].merge(pd_summaries[sum_cols], on=join_cols, how='inner')

In [36]:
pd_matches_sum

,url,json_file,article,events,summary_events
0,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,A goal and an assist from Riyad Mahrez sealed ...,Attempt blocked. Ilkay Gündogan (Manchester Ci...,Attempt saved. Riyad Mahrez (Manchester City) ...
1,http://www.premierleague.com/match/38679,premier_league_2018_2019.json,A second-half double by Pierre-Emerick Aubamey...,Hand ball by Tom Heaton (Burnley). Yellow Card...,Pierre-Emerick Aubameyang (Arsenal) hits the r...
2,http://www.premierleague.com/match/38680,premier_league_2018_2019.json,Michy Batshuayi's brace helped Crystal Palace ...,Andros Townsend (Crystal Palace) wins a free k...,Andros Townsend (Crystal Palace) wins a free k...
3,http://www.premierleague.com/match/38681,premier_league_2018_2019.json,Fulham were overpowered by an impressive Newca...,Attempt missed. Jonjo Shelvey (Newcastle Unite...,Attempt missed. Jonjo Shelvey (Newcastle Unite...
4,http://www.premierleague.com/match/38682,premier_league_2018_2019.json,Chelsea confirmed a third-place finish after a...,Foul by Jorginho (Chelsea). Jamie Vardy (Leice...,Jamie Vardy (Leicester City) wins a free kick ...
...,...,...,...,...,...
4518,https://www.espn.com/soccer/report?gameId=522712,italian_serie_a_2018_2019.json,Inter Milan notched their seventh successive S...,"Corner, Inter Milan. Conceded by Cristian Rom...",Attempt missed. Ivan Perisic (Inter Milan) rig...
4519,https://www.espn.com/soccer/report?gameId=522672,italian_serie_a_2018_2019.json,Two goals from Arkadiusz Milik helped Napoli m...,"Offside, Napoli. Elseid Hysaj tries a through ...","Goal! Napoli 1, Frosinone 0. Piotr Zielinski ..."
4520,https://www.espn.com/soccer/report?gameId=522701,italian_serie_a_2018_2019.json,Inter Milan's run of seven successive league w...,Attempt blocked. Hans Hateboer (Atalanta) head...,"Corner, Atalanta. Conceded by Samir Handanovi..."
4521,https://www.espn.com/soccer/report?gameId=522667,italian_serie_a_2018_2019.json,Inter Milan rebounded from the disappointment ...,"Corner, Inter Milan. Conceded by Bram Nuytinc...",Marco D'Alessandro (Udinese) wins a free kick ...


## Evaluación

In [66]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [67]:
file_name = 'model_rank'

In [68]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [69]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=False)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_avg.pickle


In [70]:
avg_scores_dict

{'rouge-1': {'f': 0.21306391260010107,
  'p': 0.25013632311062145,
  'r': 0.21894997408278233},
 'rouge-2': {'f': 0.045199932512915535,
  'p': 0.055650475088027035,
  'r': 0.04549660749418747},
 'rouge-l': {'f': 0.19461853429268558,
  'p': 0.30136037138338223,
  'r': 0.15482676105185525}}

### Con procesado

In [71]:
file_name = 'model_rank_processed'

In [72]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [73]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=True)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_processed_avg.pickle


In [74]:
avg_scores_dict

{'rouge-1': {'f': 0.2196694167343189,
  'p': 0.2424924246842111,
  'r': 0.24196781091666808},
 'rouge-2': {'f': 0.06244400032110038,
  'p': 0.06835160223319205,
  'r': 0.06964941411290702},
 'rouge-l': {'f': 0.2521179955746187,
  'p': 0.36621056801850377,
  'r': 0.20886141399570488}}

# GBT

## Carga de datos

In [99]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1]}
num_features = ['tfidf_sum']
model_params = {
    'min_samples_leaf': [50],
    'subsample': [0.8],  
    'learning_rate': [0.1],
    'n_estimators': [80],
    'max_depth': range(5,16,4),
    'min_samples_split': range(200,1001,300)
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [100]:
ltr_train = LTRGBTTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to rouge


In [102]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [103]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/02cf14e54e'

In [104]:
rank_model.run()

Categorizing n_players...
Computing new length...
Dropping {'n_players_cat', 'score', 'sentence_ix', 'event_ix', 'sim_previous_3', 'sim_previous_1', 'advantage', 'url', 'length', 'equalize', 'players_importance', 'total_length', 'sim_previous_5', 'json_file', 'n_stop', 'n_players', 'position'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/02cf14e54e/summaries.csv


In [105]:
pd_summaries = rank_model.read_summaries()

Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/02cf14e54e/summaries.csv


In [106]:
articles_path = '{}articles_events.csv'.format(CSV_DATA_PATH)

In [107]:
pd_matches = pd.read_csv(articles_path)

In [108]:
join_cols = ['url']
match_cols = join_cols + ['json_file', 'article', 'events']
sum_cols = join_cols + ['summary_events']

In [109]:
pd_matches_sum = pd_matches[match_cols].merge(pd_summaries[sum_cols], on=join_cols, how='inner')

In [36]:
pd_matches_sum

,url,json_file,article,events,summary_events
0,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,A goal and an assist from Riyad Mahrez sealed ...,Attempt blocked. Ilkay Gündogan (Manchester Ci...,Attempt saved. Riyad Mahrez (Manchester City) ...
1,http://www.premierleague.com/match/38679,premier_league_2018_2019.json,A second-half double by Pierre-Emerick Aubamey...,Hand ball by Tom Heaton (Burnley). Yellow Card...,Pierre-Emerick Aubameyang (Arsenal) hits the r...
2,http://www.premierleague.com/match/38680,premier_league_2018_2019.json,Michy Batshuayi's brace helped Crystal Palace ...,Andros Townsend (Crystal Palace) wins a free k...,Andros Townsend (Crystal Palace) wins a free k...
3,http://www.premierleague.com/match/38681,premier_league_2018_2019.json,Fulham were overpowered by an impressive Newca...,Attempt missed. Jonjo Shelvey (Newcastle Unite...,Attempt missed. Jonjo Shelvey (Newcastle Unite...
4,http://www.premierleague.com/match/38682,premier_league_2018_2019.json,Chelsea confirmed a third-place finish after a...,Foul by Jorginho (Chelsea). Jamie Vardy (Leice...,Jamie Vardy (Leicester City) wins a free kick ...
...,...,...,...,...,...
4518,https://www.espn.com/soccer/report?gameId=522712,italian_serie_a_2018_2019.json,Inter Milan notched their seventh successive S...,"Corner, Inter Milan. Conceded by Cristian Rom...",Attempt missed. Ivan Perisic (Inter Milan) rig...
4519,https://www.espn.com/soccer/report?gameId=522672,italian_serie_a_2018_2019.json,Two goals from Arkadiusz Milik helped Napoli m...,"Offside, Napoli. Elseid Hysaj tries a through ...","Goal! Napoli 1, Frosinone 0. Piotr Zielinski ..."
4520,https://www.espn.com/soccer/report?gameId=522701,italian_serie_a_2018_2019.json,Inter Milan's run of seven successive league w...,Attempt blocked. Hans Hateboer (Atalanta) head...,"Corner, Atalanta. Conceded by Samir Handanovi..."
4521,https://www.espn.com/soccer/report?gameId=522667,italian_serie_a_2018_2019.json,Inter Milan rebounded from the disappointment ...,"Corner, Inter Milan. Conceded by Bram Nuytinc...",Marco D'Alessandro (Udinese) wins a free kick ...


## Evaluación

In [110]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [111]:
file_name = 'model_rank_gbt'

In [112]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [114]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=False)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt_avg.pickle


In [115]:
avg_scores_dict

{'rouge-1': {'f': 0.20276205811977818,
  'p': 0.20719857786520804,
  'r': 0.23951184236461787},
 'rouge-2': {'f': 0.03983985919013643,
  'p': 0.042021128445087895,
  'r': 0.04656215404694944},
 'rouge-l': {'f': 0.1970217252137998,
  'p': 0.2958964281227826,
  'r': 0.15865173221948106}}

### Con procesado

In [122]:
file_name = 'model_rank_gbt_processed'

In [123]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [124]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=True)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt_processed_avg.pickle


In [119]:
avg_scores_dict

{'rouge-1': {'f': 0.1862690636642679,
  'p': 0.18121337137449792,
  'r': 0.23654912956406168},
 'rouge-2': {'f': 0.0541520771717992,
  'p': 0.05196353708804493,
  'r': 0.07016734074984458},
 'rouge-l': {'f': 0.24382403170381378,
  'p': 0.346439828049648,
  'r': 0.20424213646524558}}

# SVM

## Carga de datos

In [5]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'rouge',
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}

model_params = {}

In [6]:
ltr_train = LTRSVMTFTrain(model_params=model_params, ltr_params=ltr_params)

Setting target metric to rouge
Setting mode to tfidf


In [7]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [10]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/9c2f95cd9b'

In [ ]:
score_df = rank_model.get_scores_df()

In [11]:
rank_model.run()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [8]:
model = rank_model.ltr.read_model()

In [22]:
rank_model.ltr.ltr

In [8]:
features_target = rank_model.load_data()

In [18]:
type(features_target)

tuple

In [20]:
x = features_target[0]

In [ ]:
y_pred = model.predict(x)

In [14]:
y = features_target[1]

In [15]:
y

,event_ix,sentence_ix,score,url,json_file
0,0,4,0.153846,http://www.premierleague.com/match/38678,premier_league_2018_2019.json
1,1,3,0.181818,http://www.premierleague.com/match/38678,premier_league_2018_2019.json
2,2,2,0.222222,http://www.premierleague.com/match/38678,premier_league_2018_2019.json
3,3,0,0.000000,http://www.premierleague.com/match/38678,premier_league_2018_2019.json
4,4,5,0.200000,http://www.premierleague.com/match/38678,premier_league_2018_2019.json
...,...,...,...,...,...
423795,107,0,0.000000,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json
423796,108,3,0.133333,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json
423797,109,7,0.181818,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json
423798,110,1,0.090909,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json


In [105]:
pd_summaries = rank_model.read_summaries()

Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/02cf14e54e/summaries.csv


In [106]:
articles_path = '{}articles_events.csv'.format(CSV_DATA_PATH)

In [107]:
pd_matches = pd.read_csv(articles_path)

In [108]:
join_cols = ['url']
match_cols = join_cols + ['json_file', 'article', 'events']
sum_cols = join_cols + ['summary_events']

In [109]:
pd_matches_sum = pd_matches[match_cols].merge(pd_summaries[sum_cols], on=join_cols, how='inner')

In [36]:
pd_matches_sum

,url,json_file,article,events,summary_events
0,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,A goal and an assist from Riyad Mahrez sealed ...,Attempt blocked. Ilkay Gündogan (Manchester Ci...,Attempt saved. Riyad Mahrez (Manchester City) ...
1,http://www.premierleague.com/match/38679,premier_league_2018_2019.json,A second-half double by Pierre-Emerick Aubamey...,Hand ball by Tom Heaton (Burnley). Yellow Card...,Pierre-Emerick Aubameyang (Arsenal) hits the r...
2,http://www.premierleague.com/match/38680,premier_league_2018_2019.json,Michy Batshuayi's brace helped Crystal Palace ...,Andros Townsend (Crystal Palace) wins a free k...,Andros Townsend (Crystal Palace) wins a free k...
3,http://www.premierleague.com/match/38681,premier_league_2018_2019.json,Fulham were overpowered by an impressive Newca...,Attempt missed. Jonjo Shelvey (Newcastle Unite...,Attempt missed. Jonjo Shelvey (Newcastle Unite...
4,http://www.premierleague.com/match/38682,premier_league_2018_2019.json,Chelsea confirmed a third-place finish after a...,Foul by Jorginho (Chelsea). Jamie Vardy (Leice...,Jamie Vardy (Leicester City) wins a free kick ...
...,...,...,...,...,...
4518,https://www.espn.com/soccer/report?gameId=522712,italian_serie_a_2018_2019.json,Inter Milan notched their seventh successive S...,"Corner, Inter Milan. Conceded by Cristian Rom...",Attempt missed. Ivan Perisic (Inter Milan) rig...
4519,https://www.espn.com/soccer/report?gameId=522672,italian_serie_a_2018_2019.json,Two goals from Arkadiusz Milik helped Napoli m...,"Offside, Napoli. Elseid Hysaj tries a through ...","Goal! Napoli 1, Frosinone 0. Piotr Zielinski ..."
4520,https://www.espn.com/soccer/report?gameId=522701,italian_serie_a_2018_2019.json,Inter Milan's run of seven successive league w...,Attempt blocked. Hans Hateboer (Atalanta) head...,"Corner, Atalanta. Conceded by Samir Handanovi..."
4521,https://www.espn.com/soccer/report?gameId=522667,italian_serie_a_2018_2019.json,Inter Milan rebounded from the disappointment ...,"Corner, Inter Milan. Conceded by Bram Nuytinc...",Marco D'Alessandro (Udinese) wins a free kick ...


## Evaluación

In [110]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [111]:
file_name = 'model_rank_gbt'

In [112]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [114]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=False)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt_avg.pickle


In [115]:
avg_scores_dict

{'rouge-1': {'f': 0.20276205811977818,
  'p': 0.20719857786520804,
  'r': 0.23951184236461787},
 'rouge-2': {'f': 0.03983985919013643,
  'p': 0.042021128445087895,
  'r': 0.04656215404694944},
 'rouge-l': {'f': 0.1970217252137998,
  'p': 0.2958964281227826,
  'r': 0.15865173221948106}}

### Con procesado

In [122]:
file_name = 'model_rank_gbt_processed'

In [123]:
path = '{}/summaries/rouge/{}'.format(conf.METRICS_PATH, file_name)

In [124]:
scores_dict, avg_scores_dict = evaluation.evaluate(pd_matches_sum, path,
                                                   preprocess_text=True)

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/model_rank_gbt_processed_avg.pickle


In [119]:
avg_scores_dict

{'rouge-1': {'f': 0.1862690636642679,
  'p': 0.18121337137449792,
  'r': 0.23654912956406168},
 'rouge-2': {'f': 0.0541520771717992,
  'p': 0.05196353708804493,
  'r': 0.07016734074984458},
 'rouge-l': {'f': 0.24382403170381378,
  'p': 0.346439828049648,
  'r': 0.20424213646524558}}

# Con todos los resumenes

In [75]:
%%time
evaluation.evaluate_all_summaries()

Evaluating the following summaries: ['key_events_summaries_1.csv', 'key_events_summaries_3.csv', 'key_events_summaries_4.csv', 'key_events_summaries_graph_2.csv', 'key_events_summaries_graph_3.csv', 'key_events_summaries_2.csv', 'key_events_summaries_graph_5.csv', 'key_events_summaries_graph_1.csv', 'key_events_summaries_graph_4.csv']
Evaluating key_events_summaries_1.csv
Metrics already exist
Evaluating key_events_summaries_3.csv
Metrics already exist
Evaluating key_events_summaries_4.csv
Metrics already exist
Evaluating key_events_summaries_graph_2.csv
Metrics already exist
Evaluating key_events_summaries_graph_3.csv
Metrics already exist
Evaluating key_events_summaries_2.csv
Metrics already exist
Evaluating key_events_summaries_graph_5.csv
Metrics already exist
Evaluating key_events_summaries_graph_1.csv
Metrics already exist
Evaluating key_events_summaries_graph_4.csv
Metrics already exist
CPU times: user 1.76 s, sys: 172 ms, total: 1.93 s
Wall time: 2.71 s


In [125]:
pd_metrics = evaluation.output_avg_metrics()

In [126]:
pd.pivot_table(pd_metrics, index='experiment', columns=['metric', 'metric_type'], values='value')

metric                                   rouge-1            ...   rouge-l          
metric_type                                    f         p  ...         p         r
experiment                                                  ...                    
baseline_rank                           0.217420  0.244630  ...  0.327370  0.155132
baseline_rank_processed                 0.214708  0.221810  ...  0.396917  0.208477
key_events_summaries_1                  0.152305  0.337611  ...  0.404838  0.101478
key_events_summaries_1_processed        0.195851  0.520812  ...  0.615943  0.142189
key_events_summaries_2                  0.151979  0.338229  ...  0.405996  0.101074
key_events_summaries_2_processed        0.195232  0.522998  ...  0.618418  0.141428
key_events_summaries_3                  0.152305  0.337611  ...  0.404838  0.101478
key_events_summaries_3_processed        0.195851  0.520812  ...  0.615943  0.142189
key_events_summaries_4                  0.151979  0.338229  ...  0.405996  0.101074
key_events_summaries_4_processed        0.195232  0.522998  ...  0.618418  0.141428
key_events_summaries_graph_1            0.176339  0.155282  ...  0.270450  0.157788
key_events_summaries_graph_1_processed  0.131490  0.107596  ...  0.280576  0.191649
key_events_summaries_graph_2            0.151728  0.120522  ...  0.269437  0.169490
key_events_summaries_graph_2_processed  0.122147  0.091738  ...  0.291261  0.218201
key_events_summaries_graph_3            0.176339  0.155282  ...  0.270450  0.157788
key_events_summaries_graph_3_processed  0.131490  0.107596  ...  0.280576  0.191649
key_events_summaries_graph_4            0.151728  0.120522  ...  0.269437  0.169490
key_events_summaries_graph_4_processed  0.122147  0.091738  ...  0.291261  0.218201
key_events_summaries_graph_5            0.173270  0.140934  ...  0.265995  0.171442
key_events_summaries_graph_5_processed  0.134658  0.101599  ...  0.288569  0.209462
model_rank                              0.213064  0.250136  ...  0.301360  0.154827
model_rank_gbt                          0.202762  0.207199  ...  0.295896  0.158652
model_rank_gbt_processed                0.186269  0.181213  ...  0.346440  0.204242
model_rank_processed                    0.219669  0.242492  ...  0.366211  0.208861
upper_bound                             0.105305  0.067818  ...  0.228750  0.232812

[25 rows x 9 columns]

In [14]:
evaluation.bound_metrics()

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/upper_bound.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/upper_bound_avg.pickle


In [26]:
pd_bound = evaluation.output_avg_bound()

In [27]:
pd_bound

,metric,metric_type,value
0,rouge-1,f,0.105305
1,rouge-1,p,0.067818
2,rouge-1,r,0.361353
3,rouge-2,f,0.021309
4,rouge-2,p,0.013826
5,rouge-2,r,0.073771
6,rouge-l,f,0.218348
7,rouge-l,p,0.228750
8,rouge-l,r,0.232812
